Resources used:
- https://python.langchain.com/v0.1/docs/integrations/vectorstores/milvus/
- https://milvus.io/docs/integrate_with_langchain.md

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAPI_KEY") # (Optional), if OpenAI Model is used

MODEL = "mistral" # Name of the model used by Ollama
EMBEDDING_MODEL = "nomic-embed-text"
COLLECTION_NAME = 'scouting' # Name of the Collection to be created
DIMENSION = 768 # Dimension of the embeddings

URI = 'http://localhost:19530' # Connection parameters for the Milvus Server

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

In [3]:
# Configure the prompt template that is used to ask the LLM

from langchain_core.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer from the context, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
print(prompt.format(context="Here is some context", question="Here is a question"))


Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer from the context, just say that you don't know, don't try to make up an answer.
<context>
Here is some context
</context>

<question>
Here is a question
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:


In [4]:
# Use Milvus as Vectorstore

from langchain_community.vectorstores import Milvus

connection_args = {'uri': URI }

vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name=COLLECTION_NAME,
    vector_field="embeddings",
    primary_field="id",
    auto_id=True
)


In [5]:
# Convert the vector store to a retriever
# k:2 --> Limit to 2 documents
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})
# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

# Invoke the RAG chain with a specific question and retrieve the response
query = "I need a right-back who can play consistently throughout the game. He should also be good going forward"
res = rag_chain.invoke(query)
res

" Based on the context provided, neither player is specifically mentioned as a right-back or has a consistent performance in the attacking phase. However, the second player, aged 24, who plays as a left-back, shows good defensive skills with impressive speed and consistent performance over 90 minutes. He also poses some dangerous offensive threats. Therefore, if you're looking for a right-back with similar characteristics, this player might be worth considering, although he currently plays on the left side."

In [8]:
from langchain_core.documents.base import Document

def extract_metadata(doc: Document) -> dict:
    return doc.metadata

In [9]:
# How to work with meta data from our query
retrived_documents = retriever.invoke(query)
retrived_documents

[Document(page_content='\n Spielte als Stürmer und zeigte sich sehr torgefährlich.\nErzielte zwei Tore durch schnelles Umschalten und gute Positionierung.\nSein Kopfballspiel war stark und er gewann viele Luftduelle.\nTechnisch war er solide, zeigte jedoch Schwächen bei der Ballannahme unter Druck.\nDefensiv arbeitete er wenig mit, was zu Lücken im Pressing führte.\nMit 29 Jahren ist er auf seinem Leistungszenit.\n ', metadata={'id': 450360548525353427, 'scout_id': '6543', 'player_id': '9f1234567890abcdef123456', 'player_transfermarkt_id': '753468', 'grade_rating': 0.800000011920929, 'grade_potential': 0.75}),
 Document(page_content='\n Spielte erneut als Stürmer und zeigte eine hohe Torgefahr.\nErzielte ein Tor durch einen schnellen Abschluss und war ständig eine Bedrohung für die Abwehr.\nSein Kopfballspiel blieb stark und er gewann viele Zweikämpfe im Luftduell.\nTechnisch zeigte er sich verbessert, besonders bei der Ballannahme.\nDefensiv war sein Einsatz erneut begrenzt, was das P

In [10]:
metadata = extract_metadata(retrived_documents[0])
metadata

{'id': 450360548525353427,
 'scout_id': '6543',
 'player_id': '9f1234567890abcdef123456',
 'player_transfermarkt_id': '753468',
 'grade_rating': 0.800000011920929,
 'grade_potential': 0.75}

In [11]:
# Transfermarkt Link
print(f"Transfermarkt.com Link: https://www.transfermarkt.com/player-name/profil/spieler/{metadata['player_transfermarkt_id']}")

Transfermarkt.com Link: https://www.transfermarkt.com/player-name/profil/spieler/753468


In [12]:
from langchain_core.documents.base import Document

def print_texts(doc: Document):
    print(doc.page_content)

In [13]:
# get original texts
print_texts(retrived_documents[0])


 Spielte als Stürmer und zeigte sich sehr torgefährlich.
Erzielte zwei Tore durch schnelles Umschalten und gute Positionierung.
Sein Kopfballspiel war stark und er gewann viele Luftduelle.
Technisch war er solide, zeigte jedoch Schwächen bei der Ballannahme unter Druck.
Defensiv arbeitete er wenig mit, was zu Lücken im Pressing führte.
Mit 29 Jahren ist er auf seinem Leistungszenit.
 


In [14]:
irrelevant_query = "Ich will einen Kuchen backen. Welche Rezepte kannst du mir vorschlagen?"
res = rag_chain.invoke(irrelevant_query)
res

" I'm an AI and don't have the ability to bake a cake, but here are some popular cake recipes based on search frequency:\n\n1. Chocolate Cake (Very popular)\n   - Preparation Time: Approximately 1 hour\n   - Servings: 8-12 slices\n   - Ingredients: Flour, Sugar, Unsweetened Cocoa Powder, Baking Powder, Salt, Milk, Vegetable Oil, Eggs, and Vanilla Extract.\n\n2. Vanilla Cake (High Frequency)\n   - Preparation Time: Approximately 1 hour\n   - Servings: 8-12 slices\n   - Ingredients: Flour, Sugar, Baking Powder, Salt, Milk, Vegetable Oil, Eggs, and Vanilla Extract.\n\n3. Carrot Cake (Moderate Frequency)\n   - Preparation Time: Approximately 1 hour and 30 minutes\n   - Servings: 8-12 slices\n   - Ingredients: Flour, Sugar, Grated Carrots, Baking Powder, Salt, Cinnamon, Eggs, Oil, and Vanilla Extract.\n\n4. Red Velvet Cake (Less Frequent)\n   - Preparation Time: Approximately 1 hour and 30 minutes\n   - Servings: 8-12 slices\n   - Ingredients: Flour, Sugar, Unsweetened Cocoa Powder, Baking 

In [15]:
# Demonstrate retrival based on metadata
other_retriver = vectorstore.as_retriever(search_kwargs={"expr": 'scout_id == "3456"'})
expr_res = other_retriver.invoke(query)
expr_res

[Document(page_content='\n Kam als Torwart zum Einsatz und zeigte eine solide Leistung.\nParierte mehrere Schüsse gekonnt und bewahrte seine Mannschaft vor einem Rückstand.\nBei hohen Bällen wirkte er jedoch unsicher und machte Fehler in der Strafraumbeherrschung.\nSein Abschlag war stark, aber das Passspiel im Aufbau eher schwach.\nMit 28 Jahren hat er sich gut entwickelt, aber es bleibt Raum für Verbesserungen.\n ', metadata={'id': 450360548525353426, 'scout_id': '3456', 'player_id': '8e1234567890abcdef123456', 'player_transfermarkt_id': '159486', 'grade_rating': 0.6000000238418579, 'grade_potential': 0.699999988079071}),
 Document(page_content='\n Als Torwart zeigte er erneut eine solide Leistung, jedoch mit Schwächen bei hohen Bällen.\nParierte mehrere schwierige Schüsse und bewahrte seine Mannschaft vor einem frühen Rückstand.\nSeine Unsicherheiten bei der Strafraumbeherrschung blieben bestehen, was zu unnötigen Risiken führte.\nSein Passspiel war diesmal besser, insbesondere bei 